# aDDM_gridSearch Script
## Using z-scored values
## DISLIKE frame opposite


Implement the iterative step for the search of optimal parameter sets. Use of aDDM_pta_mla script from addm_toolbox (https://github.com/goptavares/aDDM-Toolbox)

In [1]:
from __future__ import absolute_import
import argparse
import addm_toolbox 
import numpy as np
import pandas as pd
import os

In [2]:
addm_toolbox

<module 'addm_toolbox' from '/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/__init__.py'>

In [3]:
addm_toolbox.addm_pta_mle_cont_lastSim

<module 'addm_toolbox.addm_pta_mle_cont_lastSim' from '/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_pta_mle_cont_lastSim.py'>

### + Free parameters in the model

#####    d : controls speed of signal integration (Free DDM parameter)
#####   sigma: standard deviation of normal distribution (Free DDM parameter)
#####   theta: controls attentional bias (Free aDDM parameter)

### + Inputs for toolobox addm_pta_mle script 
##### range_d,range_sigma,range_theta : range to define the grid for parameter search
##### trials_file_name : filepath with trial conditions (to be used for simulations)
##### expdata_file_name :  filepath with trial info [participant, trial number, RT, response, values]
##### fixations_file_name : filepath with fixations info [participant, trial num, fixation_id, fixation_time]
##### Extra parameters: trials_per_subject, simulations_per_condition, subject_ids (number of the subjects to be included for adjusting the parameters, num_threads (parallel processing),save_simulations, save_figures, verbose (deliver reports)

###  + Initial grid selection:
##### range_d = [0.003, 0.006, 0.009] ; range_sigma = [0.03, 0.06, 0.09] ; range_theta = [0.3, 0.5, 0.7]

### + Iteration (adjustment to a new grid) 

#### ** aDDM grid iterations using version of the script that does not transform item values **

## LESS INV condition

In [4]:
condition_type_name = "Dislike_Inv_zScore_31"
toSaveFolder = 'aDDM_simulations/'+condition_type_name+'/'

# Initial grid selection
#range_d = [0.003, 0.006, 0.009]
#range_sigma = [0.03, 0.06, 0.09]
#range_theta = [0.3, 0.5, 0.7]

#LEO params
## range_d = [0.001, 0.002,0.003]
## range_sigma = [0.01,0.03 ,0.05]
## range_theta = [0.3, 0.5, 0.8]



range_d = [0.001, 0.005,0.01]
range_sigma = [0.01,0.05 ,0.1]
range_theta = [0.1, 0.5, 0.9]


#data files definition
trials_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/trial_conditions_parcode_zScore.csv'
expdata_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/aDDM_expdata_FF2018_Dislike_zScore_31_Inv.csv'
fixations_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/aDDM_Fix_FF2018_Dislike_YesBin_31.csv'

trials_per_subject = 120
simulations_per_condition = 200


subjectIds=[] ## All participants


#Add valueDiffs for plotting
# For unbinned case
#l = range(-300,301,5)
#l = [x * 0.01 for x in l]

# For bin case
l = np.arange(-6,7,1)


#Give a name to the folder to store the simulations
if not os.path.exists('aDDM_simulations/'+condition_type_name):
    os.makedirs('aDDM_simulations/'+condition_type_name)

iter_num = 1
iteration_data = []
current_min_nll = 0
previous_min_nll = np.Infinity
#previous_nll_improve = 0

isLastIter = False
limitIter = 5
while ((np.abs(current_min_nll - previous_min_nll) > (previous_min_nll * 1/100)) and ((current_min_nll-previous_min_nll) < 0)) or iter_num <= limitIter:
    print ("----------------  Iteration " + str(iter_num) +" ------------------")
    
    if iter_num > limitIter: 
        isLastIter = True
    
    if iter_num > 1:  
        delta_d = range_d[1]-range_d[0]
        delta_sigma = range_sigma[1] - range_sigma[0]
        delta_theta = range_theta[1] - range_theta[0]
        
        range_d = [new_d - delta_d/2 , new_d ,new_d + delta_d/2]
        range_sigma =[new_sigma - delta_sigma/2 , new_sigma ,new_sigma + delta_sigma/2]
        range_theta =[new_theta - delta_theta/2 , new_theta ,new_theta + delta_theta/2]
    
    # control for the limits 0 < theta < 1 
    if range_theta[0]<0:
        range_theta[0] = 0 
    if range_theta[2]>1:
        range_theta[2] = 1
    
    # control for the limits 0 < d 
    if range_d[0]<0:
        range_d[0] = 0   
    
 #run the toolbox
    params, NLL = addm_toolbox.addm_pta_mle_cont_lastSim.main(range_d, range_sigma, range_theta,
                       trials_file_name, 
                       expdata_file_name,
                       fixations_file_name, 
                       trials_per_subject,simulations_per_condition, subjectIds,
                       9,saveSimulations=True, saveFigures=True,
                       verbose = True, saveFolder = toSaveFolder, genSimulations = isLastIter)
    
    # Update the NLL values for this iteration
#    previous_nnl_improve = abs(NLL - previous_min_nll)
    previous_min_nll = current_min_nll
    current_min_nll = NLL
    
    #get the new values
    new_d = params[0]
    new_sigma = params[1]
    new_theta = params[2]
    
    iteration_data.append([iter_num,new_d, new_sigma, new_theta,current_min_nll]) 

    iter_num = iter_num + 1
           
           
isLastIter
           
iteration_dataf=pd.DataFrame(iteration_data,columns=['Iteration', 'd', 'sigma', 'theta', 'min_NLL'])
print ("Finished iterations!!")

----------------  Iteration 1 ------------------
Loading experimental data...
Starting grid search...
Computing likelihoods for model (0.001, 0.01, 0.1, 1)...
Computing likelihoods for model (0.001, 0.01, 0.5, 1)...
Computing likelihoods for model (0.001, 0.01, 0.9, 1)...
Computing likelihoods for model (0.001, 0.05, 0.1, 1)...
Computing likelihoods for model (0.001, 0.05, 0.5, 1)...
Computing likelihoods for model (0.001, 0.05, 0.9, 1)...
Computing likelihoods for model (0.001, 0.1, 0.1, 1)...
Computing likelihoods for model (0.001, 0.1, 0.5, 1)...
Computing likelihoods for model (0.001, 0.1, 0.9, 1)...
Computing likelihoods for model (0.005, 0.01, 0.1, 1)...
Computing likelihoods for model (0.005, 0.01, 0.5, 1)...
Computing likelihoods for model (0.005, 0.01, 0.9, 1)...
Computing likelihoods for model (0.005, 0.05, 0.1, 1)...
Computing likelihoods for model (0.005, 0.05, 0.5, 1)...
Computing likelihoods for model (0.005, 0.05, 0.9, 1)...
Computing likelihoods for model (0.005, 0.1, 0

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:285: RuntimeWarning: invalid value encountered in true_divide
  prStatesNew = prStatesNew * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:286: RuntimeWarning: invalid value encountered in double_scalars
  tempUpCross = tempUpCross * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:287: RuntimeWarning: invalid value encountered in double_scalars
  tempDownCross = tempDownCross * sumIn / sumCurrent


Computing likelihoods for model (0.01, 0.01, 0.9, 1)...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:285: RuntimeWarning: invalid value encountered in true_divide
  prStatesNew = prStatesNew * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:286: RuntimeWarning: invalid value encountered in double_scalars
  tempUpCross = tempUpCross * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:287: RuntimeWarning: invalid value encountered in double_scalars
  tempDownCross = tempDownCross * sumIn / sumCurrent


Computing likelihoods for model (0.01, 0.05, 0.1, 1)...
Computing likelihoods for model (0.01, 0.05, 0.5, 1)...
Computing likelihoods for model (0.01, 0.05, 0.9, 1)...
Computing likelihoods for model (0.01, 0.1, 0.1, 1)...
Computing likelihoods for model (0.01, 0.1, 0.5, 1)...
Computing likelihoods for model (0.01, 0.1, 0.9, 1)...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_pta_mle_cont_lastSim.py:133: RuntimeWarning: divide by zero encountered in log
  NLL[model.params] = - np.sum(np.log(likelihoods[model.params]))


Finished grid search!
Optimal d: 0.001
Optimal sigma: 0.05
Optimal theta: 0.9
Min NLL: 13094.441806697232
True
----------------  Iteration 2 ------------------
Loading experimental data...
Starting grid search...
Computing likelihoods for model (0, 0.030000000000000002, 0.7, 1)...
Computing likelihoods for model (0, 0.030000000000000002, 0.9, 1)...
Computing likelihoods for model (0, 0.030000000000000002, 1, 1)...
Computing likelihoods for model (0, 0.05, 0.7, 1)...
Computing likelihoods for model (0, 0.05, 0.9, 1)...
Computing likelihoods for model (0, 0.05, 1, 1)...
Computing likelihoods for model (0, 0.07, 0.7, 1)...
Computing likelihoods for model (0, 0.07, 0.9, 1)...
Computing likelihoods for model (0, 0.07, 1, 1)...
Computing likelihoods for model (0.001, 0.030000000000000002, 0.7, 1)...
Computing likelihoods for model (0.001, 0.030000000000000002, 0.9, 1)...
Computing likelihoods for model (0.001, 0.030000000000000002, 1, 1)...
Computing likelihoods for model (0.001, 0.05, 0.7, 

In [5]:
## simulate last iteration

trials_file_name2 = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/trial_conditions_parcode_zScore.csv'


params, NLL = addm_toolbox.addm_pta_mle_cont_lastSim.main(range_d, range_sigma, range_theta,
                   trials_file_name2, 
                   expdata_file_name,
                   fixations_file_name, 
                   trials_per_subject,simulations_per_condition, subjectIds,
                   9,saveSimulations=True, saveFigures=True,
                   verbose = True, saveFolder = toSaveFolder, genSimulations = True)
    

Loading experimental data...
Starting grid search...
Computing likelihoods for model (0.000875, 0.0475, 0.975, 1)...
Computing likelihoods for model (0.000875, 0.0475, 1, 1)...
Computing likelihoods for model (0.000875, 0.0475, 1, 1)...
Computing likelihoods for model (0.000875, 0.05, 0.975, 1)...
Computing likelihoods for model (0.000875, 0.05, 1, 1)...
Computing likelihoods for model (0.000875, 0.05, 1, 1)...
Computing likelihoods for model (0.000875, 0.052500000000000005, 0.975, 1)...
Computing likelihoods for model (0.000875, 0.052500000000000005, 1, 1)...
Computing likelihoods for model (0.000875, 0.052500000000000005, 1, 1)...
Computing likelihoods for model (0.001, 0.0475, 0.975, 1)...
Computing likelihoods for model (0.001, 0.0475, 1, 1)...
Computing likelihoods for model (0.001, 0.0475, 1, 1)...
Computing likelihoods for model (0.001, 0.05, 0.975, 1)...
Computing likelihoods for model (0.001, 0.05, 1, 1)...
Computing likelihoods for model (0.001, 0.05, 1, 1)...
Computing likel

In [6]:
iteration_dataf.to_csv(toSaveFolder + 'ParameterIterations_zScored.csv',index = True)

In [7]:
iteration_dataf

,Iteration,d,sigma,theta,min_NLL
0,1,0.001000,0.0500,0.9,13094.441807
1,2,0.001000,0.0500,1.0,13091.779750
2,3,0.001000,0.0500,1.0,13091.779750
3,4,0.001000,0.0500,1.0,13091.779750
4,5,0.001125,0.0475,1.0,13086.172338


*** 

## Second attempt

## LESS INV condition

In [6]:
condition_type_name = "Dislike_Inv_zScore_31_v2"
toSaveFolder = 'aDDM_simulations/'+condition_type_name+'/'

# Initial grid selection
#range_d = [0.003, 0.006, 0.009]
#range_sigma = [0.03, 0.06, 0.09]
#range_theta = [0.3, 0.5, 0.7]

#LEO params
## range_d = [0.001, 0.002,0.003]
## range_sigma = [0.01,0.03 ,0.05]
## range_theta = [0.3, 0.5, 0.8]



range_d = [0.001, 0.005,0.01]
range_sigma = [0.01,0.05 ,0.1]
range_theta = [0.1, 0.5, 0.9]


#data files definition
trials_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/trial_conditions_parcode_zScore.csv'
expdata_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/aDDM_expdata_FF2018_Dislike_zScore_31_Inv.csv'
fixations_file_name = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/aDDM_Fix_FF2018_Dislike_YesBin_31.csv'

trials_per_subject = 120
simulations_per_condition = 200


subjectIds=[] ## All participants


#Add valueDiffs for plotting
# For unbinned case
#l = range(-300,301,5)
#l = [x * 0.01 for x in l]

# For bin case
l = np.arange(-6,7,1)


#Give a name to the folder to store the simulations
if not os.path.exists('aDDM_simulations/'+condition_type_name):
    os.makedirs('aDDM_simulations/'+condition_type_name)

iter_num = 1
iteration_data = []
current_min_nll = 0
previous_min_nll = np.Infinity
#previous_nll_improve = 0

isLastIter = False
limitIter = 5
while ((np.abs(current_min_nll - previous_min_nll) > (previous_min_nll * 1/100)) and ((current_min_nll-previous_min_nll) < 0)) or iter_num <= limitIter:
    print ("----------------  Iteration " + str(iter_num) +" ------------------")
    print ("range theta" + str(range_theta))
    
    if iter_num > limitIter: 
        isLastIter = True
    
    if iter_num > 1:  
        delta_d = range_d[1]-range_d[0]
        delta_sigma = range_sigma[1] - range_sigma[0]
        delta_theta = range_theta[1] - range_theta[0]
        
        range_d = [new_d - delta_d/2 , new_d ,new_d + delta_d/2]
        range_sigma =[new_sigma - delta_sigma/2 , new_sigma ,new_sigma + delta_sigma/2]
        range_theta =[new_theta - delta_theta/2 , new_theta ,new_theta + delta_theta/2]
    
    # control for the limits 0 < theta < 1 
    if range_theta[0]<0:
        range_theta[0] = 0 
    if range_theta[2]>1:
        range_theta[2] = 1
    
    # control for the limits 0 < d 
    if range_d[0]<0:
        range_d[0] = 0   
    
 #run the toolbox
    params, NLL = addm_toolbox.addm_pta_mle_cont_lastSim.main(range_d, range_sigma, range_theta,
                       trials_file_name, 
                       expdata_file_name,
                       fixations_file_name, 
                       trials_per_subject,simulations_per_condition, subjectIds,
                       9,saveSimulations=True, saveFigures=True,
                       verbose = True, saveFolder = toSaveFolder, genSimulations = isLastIter)
    
    # Update the NLL values for this iteration
#    previous_nnl_improve = abs(NLL - previous_min_nll)
    previous_min_nll = current_min_nll
    current_min_nll = NLL
    
    #get the new values
    new_d = params[0]
    new_sigma = params[1]
    new_theta = params[2]
    
    iteration_data.append([iter_num,new_d, new_sigma, new_theta,current_min_nll]) 

    iter_num = iter_num + 1
           
           
isLastIter
           
iteration_dataf=pd.DataFrame(iteration_data,columns=['Iteration', 'd', 'sigma', 'theta', 'min_NLL'])
print ("Finished iterations!!")

----------------  Iteration 1 ------------------
range theta[0.1, 0.5, 0.9]
Loading experimental data...
Starting grid search...
Computing likelihoods for model (0.001, 0.01, 0.1, 1)...
Computing likelihoods for model (0.001, 0.01, 0.5, 1)...
Computing likelihoods for model (0.001, 0.01, 0.9, 1)...
Computing likelihoods for model (0.001, 0.05, 0.1, 1)...
Computing likelihoods for model (0.001, 0.05, 0.5, 1)...
Computing likelihoods for model (0.001, 0.05, 0.9, 1)...
Computing likelihoods for model (0.001, 0.1, 0.1, 1)...
Computing likelihoods for model (0.001, 0.1, 0.5, 1)...
Computing likelihoods for model (0.001, 0.1, 0.9, 1)...
Computing likelihoods for model (0.005, 0.01, 0.1, 1)...
Computing likelihoods for model (0.005, 0.01, 0.5, 1)...
Computing likelihoods for model (0.005, 0.01, 0.9, 1)...
Computing likelihoods for model (0.005, 0.05, 0.1, 1)...
Computing likelihoods for model (0.005, 0.05, 0.5, 1)...
Computing likelihoods for model (0.005, 0.05, 0.9, 1)...
Computing likelihoo

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:285: RuntimeWarning: invalid value encountered in true_divide
  prStatesNew = prStatesNew * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:286: RuntimeWarning: invalid value encountered in double_scalars
  tempUpCross = tempUpCross * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:287: RuntimeWarning: invalid value encountered in double_scalars
  tempDownCross = tempDownCross * sumIn / sumCurrent


Computing likelihoods for model (0.01, 0.01, 0.9, 1)...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:285: RuntimeWarning: invalid value encountered in true_divide
  prStatesNew = prStatesNew * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:286: RuntimeWarning: invalid value encountered in double_scalars
  tempUpCross = tempUpCross * sumIn / sumCurrent
/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_cont.py:287: RuntimeWarning: invalid value encountered in double_scalars
  tempDownCross = tempDownCross * sumIn / sumCurrent


Computing likelihoods for model (0.01, 0.05, 0.1, 1)...
Computing likelihoods for model (0.01, 0.05, 0.5, 1)...
Computing likelihoods for model (0.01, 0.05, 0.9, 1)...
Computing likelihoods for model (0.01, 0.1, 0.1, 1)...
Computing likelihoods for model (0.01, 0.1, 0.5, 1)...
Computing likelihoods for model (0.01, 0.1, 0.9, 1)...


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/addm_toolbox/addm_pta_mle_cont_lastSim.py:133: RuntimeWarning: divide by zero encountered in log
  NLL[model.params] = - np.sum(np.log(likelihoods[model.params]))


Finished grid search!
Optimal d: 0.001
Optimal sigma: 0.05
Optimal theta: 0.9
Min NLL: 13094.441806697232
True
----------------  Iteration 2 ------------------
range theta[0.1, 0.5, 0.9]
Loading experimental data...
Starting grid search...
Computing likelihoods for model (0, 0.030000000000000002, 0.7, 1)...
Computing likelihoods for model (0, 0.030000000000000002, 0.9, 1)...
Computing likelihoods for model (0, 0.030000000000000002, 1, 1)...
Computing likelihoods for model (0, 0.05, 0.7, 1)...
Computing likelihoods for model (0, 0.05, 0.9, 1)...
Computing likelihoods for model (0, 0.05, 1, 1)...
Computing likelihoods for model (0, 0.07, 0.7, 1)...
Computing likelihoods for model (0, 0.07, 0.9, 1)...
Computing likelihoods for model (0, 0.07, 1, 1)...
Computing likelihoods for model (0.001, 0.030000000000000002, 0.7, 1)...
Computing likelihoods for model (0.001, 0.030000000000000002, 0.9, 1)...
Computing likelihoods for model (0.001, 0.030000000000000002, 1, 1)...
Computing likelihoods fo

In [7]:
## simulate last iteration

trials_file_name2 = '/Users/pradyumna/Documents/GiTs/FF2018/InputData_aDDM/trial_conditions_parcode_zScore.csv'


params, NLL = addm_toolbox.addm_pta_mle_cont_lastSim.main(range_d, range_sigma, range_theta,
                   trials_file_name2, 
                   expdata_file_name,
                   fixations_file_name, 
                   trials_per_subject,simulations_per_condition, subjectIds,
                   9,saveSimulations=True, saveFigures=True,
                   verbose = True, saveFolder = toSaveFolder, genSimulations = True)
    

Loading experimental data...
Starting grid search...
Computing likelihoods for model (0.000875, 0.0475, 0.975, 1)...
Computing likelihoods for model (0.000875, 0.0475, 1, 1)...
Computing likelihoods for model (0.000875, 0.0475, 1, 1)...
Computing likelihoods for model (0.000875, 0.05, 0.975, 1)...
Computing likelihoods for model (0.000875, 0.05, 1, 1)...
Computing likelihoods for model (0.000875, 0.05, 1, 1)...
Computing likelihoods for model (0.000875, 0.052500000000000005, 0.975, 1)...
Computing likelihoods for model (0.000875, 0.052500000000000005, 1, 1)...
Computing likelihoods for model (0.000875, 0.052500000000000005, 1, 1)...
Computing likelihoods for model (0.001, 0.0475, 0.975, 1)...
Computing likelihoods for model (0.001, 0.0475, 1, 1)...
Computing likelihoods for model (0.001, 0.0475, 1, 1)...
Computing likelihoods for model (0.001, 0.05, 0.975, 1)...
Computing likelihoods for model (0.001, 0.05, 1, 1)...
Computing likelihoods for model (0.001, 0.05, 1, 1)...
Computing likel

In [8]:
iteration_dataf.to_csv(toSaveFolder + 'ParameterIterations_zScored.csv',index = True)

In [9]:
iteration_dataf

,Iteration,d,sigma,theta,min_NLL
0,1,0.001000,0.0500,0.9,13094.441807
1,2,0.001000,0.0500,1.0,13091.779750
2,3,0.001000,0.0500,1.0,13091.779750
3,4,0.001000,0.0500,1.0,13091.779750
4,5,0.001125,0.0475,1.0,13086.172338
